# AVALIAÇÃO 3 - PROGRAMAÇÃO EM FINANÇAS
#### Professor: Julio Russo
#### Disciplina: Programação em Finanças
#### Turma: 2025.1
#### Alunos: Lucas dos Santos Marques, Luiz Fernando M. Meirinho , Victor A. S. de Souza e Victor Flávio P. Dornelos
#### Data: 25/06/2025

## 1 - Preparando os dados

In [14]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
from tabulate import tabulate
import time

In [15]:
# Carteira brasileira
acoes_br = ['BBAS3.SA','VALE3.SA','EMBR3.SA','RANI3.SA']
# Baixar os dados
df_br = yf.download(acoes_br,auto_adjust=True,start='2014-01-01',end='2024-12-31')['Close']

# Salvando em CSV
df_br.to_csv('carteira_brasileira.csv')

[*********************100%***********************]  4 of 4 completed


In [16]:
# Carteira EUA corrigida (apenas tickers válidos)
acoes_eua = ['AAPL', 'JPM', 'AMZN', 'KO', 'BRK-B', 'XOM']

# Baixar os dados de fechamento ajustado
df_eua = yf.download(acoes_eua, auto_adjust=True, start='2014-01-01', end='2024-12-31')['Close']

# Salvar em CSV
df_eua.to_csv('carteira_eua.csv')

[*********************100%***********************]  6 of 6 completed


## PARTE 1 - AÇÕES BRASIL

### QUESTÃO 1:
Formação de portfólio com ações brasileiras (mínimo 4 ativos). Não é necessário ter os mesmo pesos iniciais.


In [17]:
# Importando os dados
df_br = pd.read_csv('carteira_brasileira.csv', index_col='Date', parse_dates=True)
pesos_br = np.array([0.35, 0.35, 0.15, 0.15])  

### QUESTÃO 2:
Analisar retornos, volatilidade(desv.pad) dos ativos e da carteira

In [18]:
# Retornos logarítmicos diários
retornos_diarios = np.log(df_br / df_br.shift(1)).dropna()

# Indicadores individuais
retorno_anual = retornos_diarios.mean() * 252
retorno_acumulado = (df_br.iloc[-1] / df_br.iloc[0]) - 1
desvio_padrao_anual = retornos_diarios.std() * np.sqrt(252)
coef_variacao = desvio_padrao_anual / retorno_anual

In [19]:
# Tabela por ativo
tabela_ativos = pd.DataFrame({
    'Retorno Anual (%)': retorno_anual * 100,
    'Retorno Acumulado (%)': retorno_acumulado * 100,
    'Desvio Padrão Anual (%)': desvio_padrao_anual * 100,
    'Coef. de Variação': coef_variacao
})

In [20]:
# Retorno do portfólio
retorno_portfolio_diario = retornos_diarios @ pesos_br
retorno_anual_ptf = retorno_portfolio_diario.mean() * 252
retorno_acumulado_ptf = np.exp(retorno_portfolio_diario.cumsum())[-1] - 1
desvio_padrao_ptf = retorno_portfolio_diario.std() * np.sqrt(252)
coef_variacao_ptf = desvio_padrao_ptf / retorno_anual_ptf

/var/folders/wk/k6089y_d7hv_rwb2m8vbm9cr0000gn/T/ipykernel_16761/989489004.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [21]:
# Adiciona à tabela
tabela_ativos.loc['Carteira'] = [
    retorno_anual_ptf * 100,
    retorno_acumulado_ptf * 100,
    desvio_padrao_ptf * 100,
    coef_variacao_ptf
]

In [22]:
# Arredondar para melhor visualização
tabela_ativos = tabela_ativos.round(2)

In [23]:
# Taxa livre de risco (ajuste conforme o país e período)
r_f = 0.01  # Exemplo: 12% a.a. no Brasil

# Sharpe dos ativos
sharpe_ativos = (retorno_anual - r_f) / desvio_padrao_anual

# Sharpe da carteira
sharpe_portfolio = (retorno_anual_ptf - r_f) / desvio_padrao_ptf

# Adiciona à tabela
tabela_ativos['Sharpe Ratio'] = sharpe_ativos
tabela_ativos.loc['Carteira', 'Sharpe Ratio'] = sharpe_portfolio

# Exibe com tabulate
from tabulate import tabulate
print("\n📊 Indicadores com Sharpe Ratio:\n")
print(tabulate(tabela_ativos.round(2), headers='keys', tablefmt='fancy_grid'))


📊 Indicadores com Sharpe Ratio:

╒══════════╤═════════════════════╤═════════════════════════╤═══════════════════════════╤═════════════════════╤════════════════╕
│          │   Retorno Anual (%) │   Retorno Acumulado (%) │   Desvio Padrão Anual (%) │   Coef. de Variação │   Sharpe Ratio │
╞══════════╪═════════════════════╪═════════════════════════╪═══════════════════════════╪═════════════════════╪════════════════╡
│ BBAS3.SA │               12.9  │                  305.52 │                     41.51 │                3.22 │           0.29 │
├──────────┼─────────────────────┼─────────────────────────┼───────────────────────────┼─────────────────────┼────────────────┤
│ EMBR3.SA │               10.39 │                  208.57 │                     43.18 │                4.16 │           0.22 │
├──────────┼─────────────────────┼─────────────────────────┼───────────────────────────┼─────────────────────┼────────────────┤
│ RANI3.SA │               10.97 │                  228.81 │          

### QUESTÃO 3:
Construção dos gráficos em plotly de retorno e gráfico PIE das participações relativas das ações no portfólio.

In [24]:
df_br

,BBAS3.SA,EMBR3.SA,RANI3.SA,VALE3.SA
Date,,,,
2014-01-02,5.937052,18.191263,1.919264,16.673845
2014-01-03,5.998895,18.115461,1.919264,16.314604
2014-01-06,5.909838,18.096516,1.981176,16.328970
2014-01-07,5.808412,18.286005,1.981176,15.926612
2014-01-08,5.692148,18.380749,1.981176,16.036783
...,...,...,...,...
2024-12-20,23.986444,54.903511,6.505034,52.568302
2024-12-23,23.827066,55.492905,6.440260,52.789661
2024-12-26,24.105978,56.182198,6.412501,52.943653


In [25]:
# Criar DataFrame
df_pie = pd.DataFrame({
    'Ativo': acoes_br,
    'Peso': pesos_br
})

In [26]:
# Cores personalizadas
cores_personalizadas = {
    'BBAS3': 'gold',         # amarelo
    'VALE3': 'red',          # vermelho
    'EMBR3': 'darkblue',     # azul escuro
    'RANI3': 'green'         # verde
}


In [27]:
# Gráfico de pizza
fig = px.pie(df_pie,
             names='Ativo',
             values='Peso',
             title='Participações Relativas das Ações no Portfólio',
             color='Ativo',
             color_discrete_map=cores_personalizadas)

fig.update_traces(textinfo='percent+label', hole=0)  # hole=0 remove o buraco

fig.show()